You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Title: The Future of Artificial Intelligence: Latest Trends and Key Players

Introduction:
- Brief overview of the current state of Artificial Intelligence (AI) technology
- Importance of staying updated on the latest trends and key players in the AI industry
- Purpose of the article: to educate and inform the audience on the advancements in AI

Key Points:
1. Latest Trends in Artificial Intelligence:
- Machine learning and deep learning advancements
- Ethical considerations in AI development
- Integration o

I now can give a great answer

Final Answer:
# The Future of Artificial Intelligence: Latest Trends and Key Players

**Introduction:**
Artificial Intelligence (AI) has rapidly evolved in recent years, revolutionizing various industries and impacting our daily lives. It is crucial to stay updated on the latest trends and key players in the AI industry to understand the advancements shaping our future. This article aims to educate and inform the audience on the exciting developments in AI technology.

**Latest Trends in Artificial Intelligence:**
In the realm of AI, machine learning and deep learning have seen significant advancements, allowing machines to learn from data and perform tasks without explicit programming. Ethical considerations in AI development have also come to the forefront, highlighting the importance of responsible AI implementation. Moreover, AI integration in industries such as healthcare, finance, and transportation is transforming processes and enhancing efficiency

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# The Future of Artificial Intelligence: Latest Trends and Key Players

**Introduction:**
Artificial Intelligence (AI) has rapidly evolved in recent years, revolutionizing various industries and impacting our daily lives. It is crucial to stay updated on the latest trends and key players in the AI industry to understand the advancements shaping our future. This article aims to educate and inform the audience on the exciting developments in AI technology.

**Latest Trends in Artificial Intelligence:**
In the realm of AI, machine learning and deep learning have seen significant advancements, allowing machines to learn from data and perform tasks without explicit programming. Ethical considerations in AI development have also come to the forefront, highlighting the importance of responsible AI implementation. Moreover, AI integration in industries such as healthcare, finance, and transportation is transforming processes and enhancing efficiency.

**Key Players in the AI Industry:**
Leading the AI revolution are key players such as Google AI, IBM Watson, Microsoft Azure AI, Amazon Web Services AI, OpenAI, and NVIDIA. These companies are at the forefront of innovation, driving progress and pushing the boundaries of AI technology.

**Noteworthy News in Artificial Intelligence:**
Recent breakthroughs in natural language processing have enabled machines to understand and generate human language, opening up new possibilities in communication and interaction. The applications of AI in autonomous vehicles are reshaping the transportation sector, paving the way for safer and more efficient mobility. However, the impact of AI on job automation and workforce changes raises important questions about the future of work and skills needed in the AI era.

**Audience Analysis:**
For tech enthusiasts, industry professionals, and business leaders, staying updated on AI technology and understanding its impact on different industries is crucial. Navigating the complexities of AI technology and identifying trustworthy sources for AI news can be challenging, making articles like this valuable for gaining insights into the future of AI.

**Call to Action:**
I encourage readers to subscribe to a newsletter for regular updates on AI trends and news, stay informed, and engage in discussions on social media platforms about the future of AI. Links to relevant resources and articles for further reading are provided to deepen understanding and knowledge of AI advancements.

By following this comprehensive content plan, the blog article on the latest trends and key players in Artificial Intelligence will provide valuable insights and information to the target audience, ultimately achieving the goal of educating and engaging readers on the future of AI technology.

Let me know if there is anything else you would like me to add or adjust to make this blog post even more compelling and informative.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [13]:
topic = "Sports Cars"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Sports Cars.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan: Sports Cars

Outline:
I. Introduction
- Definition of sports cars
- Brief history of sports cars
- Why sports cars are popular

II. Latest Trends in Sports Cars
- Electric sports cars
- Autonomous driving technology in sports cars
- Sustainable materials in sports car manufacturing

III. Key Players in the Sports Car Industry
- Ferrari
- Lamborghini
- Porsche
- McLaren

IV. Noteworthy News in Sports Cars
- Recent advancements in sports car technology
- New sports car models released by key players
- Perfor

I now can give a great answer

Final Answer:
# The Thrilling World of Sports Cars

Sports cars have always been the epitome of high performance and sleek design, offering an exhilarating driving experience that captivates both enthusiasts and casual drivers. Dating back to the early 20th century, sports cars were specifically crafted for speed and agility, setting the foundation for the thrilling world we know today. The evolution of sports cars has led to a blend of power and precision on the road, making them a symbol of automotive excellence.

In recent years, the sports car industry has witnessed a notable shift towards sustainability and innovation, with electric sports cars gaining traction among environmentally conscious consumers seeking alternative fuel options. The integration of autonomous driving technology in some sports car models offers a glimpse into the future of automotive design, highlighting a commitment to pushing boundaries. Manufacturers are also exploring the us

In [14]:
Markdown(result)

# The Thrilling World of Sports Cars

Sports cars have always been the epitome of high performance and sleek design, offering an exhilarating driving experience that captivates both enthusiasts and casual drivers. Dating back to the early 20th century, sports cars were specifically crafted for speed and agility, setting the foundation for the thrilling world we know today. The evolution of sports cars has led to a blend of power and precision on the road, making them a symbol of automotive excellence.

In recent years, the sports car industry has witnessed a notable shift towards sustainability and innovation, with electric sports cars gaining traction among environmentally conscious consumers seeking alternative fuel options. The integration of autonomous driving technology in some sports car models offers a glimpse into the future of automotive design, highlighting a commitment to pushing boundaries. Manufacturers are also exploring the use of sustainable materials in sports car manufacturing, further solidifying the industry's dedication to eco-friendly performance vehicles.

Iconic names such as Ferrari, Lamborghini, Porsche, and McLaren continue to dominate the sports car industry, renowned for their high-quality, high-performance vehicles that redefine automotive engineering standards. With a relentless focus on innovation and precision, these key players set the benchmark for excellence in the sports car market, showcasing a commitment to pushing the limits of performance and design. The continuous advancements in sports car technology have revolutionized the driving experience, with new models boasting enhanced aerodynamics and engine performance, raising the bar for speed and handling in the industry. Performance upgrades have become more accessible, allowing drivers to customize their vehicles for optimal track and road performance, catering to a diverse audience of enthusiasts, professionals, luxury car buyers, and tech-savvy millennials passionate about cutting-edge automotive technology. Enthusiasts are encouraged to explore the world of sports cars, staying informed about the latest models and advancements in the industry through reputable sources like Automotive News, Car and Driver magazine, Top Gear website, Motor Trend magazine, and Sports Car Market magazine. The thrill of high-performance vehicles awaits, offering a blend of excitement and sophistication that continues to redefine the automotive landscape. Happy driving!

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).